#  第四章 智能体经典范式
## Motivation：为什么“重复造轮子”？
> 本章学习智能体经典范式，包括：ReAct、Plan-and-Solve、Reflection等。

目前市面过多高度抽象Agent框架，不利于理解智能体的运行原理。标准化逐渐不利于Agent的定制化。
🎯核心学习目标：
- 理解设计机制：理解ReAct、Plan-and-Solve、Reflection如何在代码层面实现。
- 暴露工程挑战：亲手处理那些被框架隐藏的难题，如模型输出解析、工具调用的重试、防止死循环等。
- 定制化能力：通过理解经典范式，能够定制化实现符合需求的智能体。

## Definition：
三种经典范式，定义了智能体组织“思考”与“行动”的不同方式：
ReAct（Reasoning and Acting）：
- 将“Reasoning”与“Acting”两个过程合并到一个循环中。
- 核心循环：不断重复Thought -> Action -> Observation 的循环。根据上一步的Observation动态调整下一步的Thought。
Plan-and-Solve：
- “三思而后行”，将任务明确解耦为两个阶段。
- 核心流程：
  1. 规划（Plan）：根据当前环境和任务，生成一个完整、清晰、分步骤的行动计划。
  2. 执行（Slove）：智能体严格按照这个（通用是静态的）计划执行。
Reflection：
- 赋予智能体“自我批判和修正”能力的范式。
- 核心循环：它引入了一个“事后（post-hoc）”的自我矫正循环：Execute -> Reflect -> Optimize。智能体先完成“初稿”后，进行自我结果的“评审”，根据“反馈”生成“修订稿”。

## Design：
### 实现基础：
环境依赖：`openai`、`python-dotenv`【包含第一章】
配置管理：`env.`文件存放`LLM_API_KEY`、`LLM_API_BASE`等配置项。
基础封装：`HelloAgentLLM`客户端类，封装OpenAI兼容接口的调用，默认用流式响应（`stream=True`）。
